# Statistics

---

### Names: [name here]

**Before you do anything else, go to File -> Save a Copy in Drive. Make any requested edits to that copy.**

This lab explores probability distributions, how they can be propogated through an equation, and how we can use estimates of uncertainties in our data to find significant emission features.

#### New Code:

This lab will introduce a number of different pieces of code, including code that can:


*   Draw random numbers from a Poisson distribution
*   Draw random numbers from a Gaussian distribution
* `==`, `!=`, `>`, `<`, `<=`, `>=`, `&`, `|`
* Select out points in a numpy array based on certain criteria. 
* Plot data points with error bars

---




In [ ]:
# Import useful packages
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline

#!pip install git+http://github.com/astropy/astroquery.git#egg=astroquery
import warnings
warnings.filterwarnings('ignore',module='astropy.io.votables.tree')
warnings.filterwarnings('ignore',message='.*unclosed..socket')

import astropy.units as u
from astropy.coordinates import SkyCoord, ICRS
from astroquery.sdss import SDSS #package that allows queries of the SDSS database

### Identifying statistically significant features

One important problem in astronomy, and in many other fields, is determining if there is a real signal in your data. For astronomy that can mean knowing whether you have detected a particular star in an image, if you have detected an absorption or emission line within a spectrum, or if you have detected a change in brightness in a light curve due to e.g. the transit of a planet. You can determine if the signal is statistically significant (i.e., the signal is highly unlikely to be cause by random fluctuations in the background noise) if you have a good estimate of the background flux and the noise. But sometimes these can be hard to measure.

Below you will explore this using the light curve (i.e., measurements of flux at different times) measured by the Kepler space telescope for a young star. The code block below reads in the light curve (make sure to upload the necessary fits file from Glow) and it populates three numpy arrays:


*   `time`: Modified Julian Date at which the observations were taken. The modified Julian Date is the number of days since midnight on Nov 17, 1858, written in decimal form.
*   `flux`: The flux during each observations
*   `flux_err`: The uncertainty in each flux observation.


In [ ]:
image = fits.open('EPIC203954898_kepler_llc.fits')
time, flux, flux_err = [], [], []
for i in image[2].data:
  time.append(i[0])
  flux.append(i[1])
  flux_err.append(i[2])
time = np.array(time)
flux = np.array(flux)
flux_err = np.array(flux_err)
image.close()

> **Q:** Plot the light curve (time vs flux) for this object. Label the axes. Print out the mean and standard deviation ($\sigma$). To the graph, add a horizontal line indicating the mean+3$\sigma$.

In [ ]:
# Plot the light curve


If we assume the noise on the flux is a Gaussian distribution, then we could use Gaussian statistics to determine the probability of each peak (or conversely, set a threshold above which any features are highly unlikely to be due to random chance). But calculating the mean and standard deviation directly from all of the data is skewed by the outbursts themselves.


What we want is to select out relatively smooth regions of the light curve, and calculate the mean and standard deviation from those regions. To do this we can take advantage of logical operators in python, and the combination of boolean arrays with numpy arrays.

If we use a statement like:

```
late = time>56965
```

the variable `late` will have the same shape as `time` but each element will be either `True` or `False`, depending on whether or not that element satisfies the logical condition. Note that this only works for numpy arrays, and will fail for lists or tuples. 

The following code prints out each element of `time`, and the corresponding element of the array `late`. To demonstrate what this looks like. For every element of `time` that is greater than `56965` the corresponding entry in `late` will be `True`, otherwise it will be `False`.

In [ ]:
late = time>56965
for i in range(len(time)):
    print(time[i],late[i])

One advantage of defining the variable `late` is that it can be passed to the variable `time` as you would any index. The result will select out the elements in `time` where `late` is True.

In [ ]:
print(time[late])

Another helpful feature of these boolean arrays is that True is equivalent to 1 while False is equivalent to 0. This means that we can quickly find out how many elements within `time` satisfy our condition by summing `late`.

In [ ]:
print('Number of elements with time>26965: ',late.sum())

This selection can also be applied to other arrays that have the same shape at `time`. For example, `flux[late]` selects out the flux values that have time>56965. In this way you can select out a range of times in the light curve, and then utilize the fluxes from those times.

In selecting out time frames, it may be helpful to use comparison operators:


*   `==` equal
*   `!=` not equal
*   `>` greater than
*   `<` less than
*   `>=` greater than or equal
*   `<=` less than or equal

and bitwise logical operators

* `&` and: Returns true if both statements are true
* `|`  or: Returns true if either statement is true

This allows us to select out multiple regions of the light curve, e.g.:

```
my_selection = ((time > 56900) & (time < 56950)) | (time > 56965)
``` 

The above code selects out those times that are between 56900 and 56950 or greater than 56965. 

>**Q:** Select out a time frame, or a set of time frames, within the light curve in which there are no obvious outbursts. Estimate the mean and standard deviation ($\sigma$) from these quiescent regions of the light curve. Plot the light curve, along with a horizontal line marking the value that is 3$\sigma$ above the mean. Any outburst that has a signal larger than this horizontal line is considered real (i.e. is not a product of noise in the data). 




In [ ]:
# Insert code here


>**Q:** What is the error on the mean that you just calculated?

**[insert answer here]**

>**Q:** There are 2863 data points in this light curve. If the signal was purely random noise, how many data points would exceed our 3 sigma threshold just based on random chance?

**[insert answer here]**

Above you estimated the uncertainty in the data by calculating the standard deviation in a region of the light curve where there was no significant signal. You can do a similar thing with images. By selecting out a region of the image that does not have any emission from stars, nebula, etc. you can estimate the average background value (which you need to subtract off from your signal) and the uncertainty. 

Go back to your code for calculating the light curve for the M101 supernova, from lab 4. 

> Copy that code and paste it into the empty code block below. You don't need to process the data again, just copy the code that reads in the processed images and calculates the flux based in the image. Processed images can be downloaded from Glow. 

Now you will modify the code to calculate the uncertainties in light curve, and use these uncertainties to tell if the flux has returned to its pre-outburst level.

> Identify a region within the images in which there is no emission from the galaxy, or any stars. This will serve as the background. For each image, calculate the median and standard deviation in this background region. This represents the typical background level and noise. 

> **Q:** What are the background levels and noise values for each image? Are they the same or different?

**[insert answer here]**

> **Q:** The noise that you calculated is the noise per pixel. But you calculate the total flux of the supernova over many pixels. If the noise per pixel is $\sigma$, what is the noise level is $N$ pixels?

**[insert answer here]**

> **Q:** What is the flux of the supernova on each day? What is the uncertainty on that flux?

**[insert answer here]**

In [ ]:
# Insert code here

To plot the data with error bars we need to use a new plotting routine:

`plt.errorbar(xvals,yvals,yerr=yerrs,fmt='.k')`

The `plt.errorbar` function will plot data with error bars. This function takes as inputs the x and y values of the data. The uncertainties in the y-direction (`yerrs`) are sent using the optional keyword `yerr`. The `fmt` keyword sets the format of the plot points; in the example above that is black small dots.

> **Q:** Plot the light curve with error bars for the M101 supernova. Add a horizontal line at the pre-supernova level. 

> **Q:** Is the final flux consistent with the pre-supernova flux level? This will be the case if the difference between the final flux level and the pre-supernoa flux level is less than three times the noise. The noise in this case is the combination of the noise from the two flux measurements ($\sigma = \sqrt{\sigma_{pre-sn}^2+\sigma_{final}^2}$, where $\sigma_{pre-sn}$ is the noise in the pre-supernova flux measurement, and $\sigma_{final}$ is the noise in the final flux measurement.)

**[insert answer here]**

In [ ]:
# Insert code here

### Probability Distributions

If we repeat an experiment many times (e.g., take many image of a star and measure its flux in each image) we will not obtain the exact same result every time. Instead we will obtain a range of different values, and the relative probability of each of these values is given by a probability distribution. Python has built-in functions that allow us to draw values from a Poisson or Gaussian distribution. This can be useful if we want to model an experiment, or estimate the relative probability of obtaining different values.

Lets first look at a Poisson distribution. Remember that the Poisson distribution is defined for integer values and is given by :

$P(x,\mu ) = \frac{\mu^x}{x!}\exp^{-\mu}$

where $\mu$ is the average value of the distribution. Poisson distributions apply to counting statistics, e.g. counting the number of photons that arrive at our detector.

Within python, we can draw a random number from a Poisson distribution. If we do this a larger number of times, and plot a histogram of the different values drawn from this distribution, we can visualize the Poisson distribution.

In [ ]:
# The next line sets the average value
mu = 2

# The next line draws 1000 random number from a Poisson distribution
poiss_data = np.random.poisson(mu,size=1000) #<- the size parameter sets the number of random numbers drawn from the given distribution

# The next lines plot a historgram of the random numbers 
p = plt.hist(poiss_data,30,histtype='step')
plt.title('Poisson Distribution')

Next, lets look at a Gaussian distribution. The formula for a Gaussian distribution is:

$P(x, \mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}}\exp^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2} $

where $\mu$ is the mean of the distribution and $\sigma$ is the standard deviation.

As with the Poisson distribution, we can draw random numbers from a Gaussian distribution. If we draw enough random numbers from this distribution, and plot a histogram of the result, we can visualize the Gaussian distribution.

In [ ]:
# Gaussian distribution
mean, std = 500, 40

# Draw 10000 random numbers from a Gaussian distribution
gauss_data = np.random.normal(mean,std,size=10000)

p = plt.hist(gauss_data,30,histtype='step')
plt.title('Gaussian Distribution')

Lets go through an exercise where we use the ability to generate random numbers.

The distance to a nearby object can be derived from the parallax by the following formula:

$d = 1/p$

where $d$ is the distance in units of parsecs and $p$ is the parallax angle, in the units of arc-seconds. As you saw in the databases lab, *Gaia* is a European space telescope that has measured the parallaxes for over one billion stars. They discourage derving the distance directly from objects with an error on the parallax of larger than 20%, but instead they suggest working directly with the parallax. Why is this?

Lets consider the star 55 Cnc. According to Gaia, it has p = 0.079427$\pm$0.000077 arc-seconds.

> **Q:** Using the formula for distance, and what you know about error propogation, calculate the distance to 55 Cnc, and its uncertainty.

**[Insert answer here]**

We can estimate the answer to the above question using random numbers. We first draw many random numbers from a Gaussian distribution with a mean and standard deviation given by the parallax measurement and its uncertainty. We then apply the distance formula to these values, and look at the the resulting distribution.

> **Q:** The code below performs this operation, but is missing important pieces, and contains some syntax errors. Fill in the missing pieces and fix the errors.

In [ ]:
p_mean = 0.079427
p_std = 0.000077

p_gauss =  #Insert code to draw 1000 random number from a Gaussian distribution with a mean of p_mean and a standard deviation of p_std
d_gauss = 1/p_gauss

# Plot a histogram of the distances
plt.hist(d_gauss,30,histtype='step'
plt.xlabel('D (pc)',fontsize=14)
plt.title('Distance to 55 Cnc',fontsize=14)

# Calculate the mean and standard deviation of the distances
mean_d = mean(d_gauss)
std_d = np.std(mean_d) 
print('Distance to 55 Cnc {:0.3f}+-{:0.3f} pc'.format(mean_d,std_d))

> **Q:** How does the mean and standard deviation estimated in the code block above compare to your direct estimate from the formula and propogation of errors?

**[Insert answer here]**

Now lets consider what would happen if the uncertainty was larger.

> **Q:** Copy the code block from above and increase the uncertainty on the parallax measurement to 0.016 arc-seconds (=20% of the parallax measurement). What are the mean and standard deviation of the distribution of distance values? Do they match what you calculate using the distance formula and the propogation of error prescription? Does the shape of the distribution of distance values look Gaussian?

**[Insert answer here]**

In [ ]:
# Insert code here


Now let's use what we have seen to estimate the uncertainty on a more complicated calculated quantity.

In lab 3 you used the distribution of redshifts for galaxies in the Virgo cluster to calculate the mass of the Virgo cluster. But what is the uncertainty on your mass estimate? You could use the propogation of errors calculation to figure this out, but you would need to know how to estimate the uncertainty on the standard deviation, and would need to account for the fact that each galaxy has a different uncertainty on its redshift.

Another way is to use the random number generators described above. The idea is as follows:

Suppose we had four galaxies with the following redshifts:
* z$_1$ = 0.0502 $\pm$ 0.001
* z$_2$ = 0.0562 $\pm$ 0.002
* z$_3$ = 0.0579 $\pm$ 0.001
* z$_4$ = 0.0584 $\pm$ 0.007

The standard deviation of the redshifts of these four galaxies is 0.0033, which implies a cluster mass of M$_{cluster}$=10.2 $\times$ 10$^{14}$ M$_{sun}$. 

Lets imagine a series of measurements of the redshifts of each galaxy. Each measurement of the redshift is a random number drawn from the Gaussian distribution with a mean set by the previous measurement and a stadard deviation set by the uncertainty.

* For galaxy 1 we draw a random number from a Gaussian distribution with a mean of 0.0502 and a standard deviation of 0.001 and record this as z$_{1a}$. 
* For galaxy 2 we draw a random number from a Gaussian distribution with a mean of 0.0562 and a standard deviation of 0.002 and record this as z$_{2a}$. 
* For galaxy 3 we draw a random number from a Gaussian distribution with a mean of 0.0579 and a standard deviation of 0.001 and record this as z$_{3a}$. 
* For galaxy 4 we draw a random number from a Gaussian distribution with a mean of 0.0584 and a standard deviation of 0.007 and record this as z$_{4a}$. 

This leaves us with four numbers (z$_{1a}$, z$_{2a}$, z$_{3a}$, z$_{4a}$), each representing a simulated measurement of the redshift for the corresponding galaxy. Suppose we do this exercise and are left with the following list of redshifts:

* z$_{1a}$ = 0.05099
* z$_{2a}$ = 0.05647
* z$_{3a}$ = 0.05770
* z$_{4a}$ = 0.05879

This distribution of redshifts has a standard deviation of $\sigma_z$ = 0.0030, which is similar to, but slightly different from our answer above. If we plug this value into our mass formula the result is M$_{cluster}$ = 8.6 $\times$ 10$^{14}$ M$_{sun}$.

If we draw another redshift from each of these four distributions we might get the following values:

* z$_{1b}$ = 0.04970
* z$_{2b}$ = 0.05803
* z$_{3b}$ = 0.05582
* z$_{4b}$ = 0.06905

which has $\sigma_z$ = 0.0070, which corresponds to M$_{cluster}$ = 46.8 $\times$ 10$^{14}$ M$_{sun}$.

Now imagine repeating this exercise 100 (or 1000 or 10000) more times. We would end up with 100 (or 1000 or 10000) values of M$_{cluster}$. As with the distance calculation above, the mean of these M$_{cluster}$ values represents our best estimate of the true value of M$_{cluster}$, while the standard deviation of these 100 (or 1000 or 10000) values represents our estimate of the uncertainty on M$_{cluster}$

Confused? That is okay. We will walk through all of this again using python code.

> **Q:** In the space below copy your SDSS SQL query for galaxies in the Virgo cluster. To this query, add that you want `zerr` in addition to `z`.

In [ ]:
### Insert SDSS SQL query here


To make our lives easier, let's write a function that returns the mass of the cluster, given a list of redshifts as an input. 

> **Q:** The outline for a function that returns the mass of the cluster (`Mcluster`) is included below. The function takes a required input that is the list of redshifts (`redshifts`) and an optional input that is the radius of the cluster (`Rcluster`, with default value of `7.5e6`). Fill in the space with the code necessary for calculating the mass of the cluster. 

In [ ]:
def  ### Set up function definition
    G = 6.67e-8 #gravitational constant in cgs units
    Msun = 1.99e33 #mass of the Sun in cgs units
    c = 2.99e10 #speed of light in cgs units
    ly = 9.461e17 # light year, in units of cm
    Mcluster =  ### Calculate Mcluster
    return Mcluster

> **Q:** In the space below, write out pseudo-code for our series of simulated mass measurements. What the code needs to do is randomly select a redshift for each galaxy in the cluster, based on its observed redshift and uncertainty, and then calculate the cluster mass based on this collection of redshifts. The process is repeated 100 (or 1000 or 10000) times, and in the end we want to know the mean and standard deviation of this distribution of cluster mass values. 

```

Insert pseudo-code here

```

> **Q:** Convert the algorithm you outlined in pseudo-code into actual code. What is the mass of the Virgo cluster, and what is the uncertainty on that mass?

In [ ]:
# Insert code here


### To turn in this lab, share the lab with me using the *Share* button in the upper right.